In [ ]:
!unzip /content/A.zip

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import random_split
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
from google.colab import drive
import os
drive.mount('/content/drive')
import numpy as np
import time
import pandas as pd
from matplotlib.image import imread

from PIL import Image
import cv2
import pandas as pd

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
import os

# List files in the directory
files = os.listdir('/content/drive/MyDrive/APS360 Project/archive/cropped_dataset/train')
print(files)


['930491_mask_3.png', '927644_mask_11.png', '930028_mask_2.png', '930028_mask_15.png', '930028_mask_3.png', '927644_mask_9.png', '930491_mask_1.png', '930028_mask_7.png', '930491_mask_8.png', '930491_mask_4.png', '930028_mask_6.png', '930028_mask_12.png', '930491_mask_12.png', '930491_mask_5.png', '930491_mask_2.png', '930491_mask_10.png', '930028_mask_8.png', '935193_mask_14.png', '934795_mask_15.png', '934795_mask_0.png', '935193_mask_5.png', '935193_mask_10.png', '934795_mask_5.png', '935193_mask_11.png', '934795_mask_4.png', '930491_mask_15.png', '930491_mask_14.png', '935193_mask_4.png', '935193_mask_7.png', '935193_mask_6.png', '935193_mask_2.png', '935193_mask_13.png', '934795_mask_14.png', '934795_mask_3.png', '934795_mask_1.png', '934795_mask_12.png', '935193_mask_8.png', '935193_mask_3.png', '934795_mask_7.png', '935193_mask_0.png', '934795_mask_6.png', '934795_mask_8.png', '935193_mask_9.png', '935193_mask_1.png', '934795_mask_11.png', '934795_mask_13.png', '934795_mask_2.pn

In [9]:
base_dir = "/content/drive/MyDrive/APS360 Project/archive/cropped_dataset"

DATA_DIR = os.path.join(base_dir, "train")
COLOR_CODES = pd.read_csv(os.path.join(base_dir, "class_dict.csv"))
annotations = pd.read_csv(os.path.join(base_dir, "cropped_proportion_train.csv"))

In [10]:
column_name = "split"
img_name = "sat_image_path"
mask_name = "mask_path"

# Extract the specific column
column_data = annotations[column_name]

train_list = []
train_mask = []
valid_list = []
valid_mask = []
test_list  = []
test_mask  = []

for i in range(len(column_data)):
  if column_data[i] == "train":
    train_list.append(annotations[img_name][i])
    train_mask.append(annotations[mask_name][i])
  elif column_data[i] == "valid":
    valid_list.append(annotations[img_name][i])
    valid_mask.append(annotations[mask_name][i])
  else:
    test_list.append(annotations[img_name][i])
    test_mask.append(annotations[mask_name][i])

In [11]:
print(len(train_list))
print(train_list)
print(len(valid_list))
print(valid_list)
print(len(test_list))
print(test_list)

9616
['train/100694_sat_0.jpg', 'train/100694_sat_1.jpg', 'train/100694_sat_2.jpg', 'train/100694_sat_3.jpg', 'train/100694_sat_4.jpg', 'train/100694_sat_5.jpg', 'train/100694_sat_6.jpg', 'train/100694_sat_7.jpg', 'train/100694_sat_8.jpg', 'train/100694_sat_9.jpg', 'train/100694_sat_10.jpg', 'train/100694_sat_11.jpg', 'train/100694_sat_12.jpg', 'train/100694_sat_13.jpg', 'train/100694_sat_14.jpg', 'train/100694_sat_15.jpg', 'train/102122_sat_0.jpg', 'train/102122_sat_1.jpg', 'train/102122_sat_2.jpg', 'train/102122_sat_3.jpg', 'train/102122_sat_4.jpg', 'train/102122_sat_5.jpg', 'train/102122_sat_6.jpg', 'train/102122_sat_7.jpg', 'train/102122_sat_8.jpg', 'train/102122_sat_9.jpg', 'train/102122_sat_10.jpg', 'train/102122_sat_11.jpg', 'train/102122_sat_12.jpg', 'train/102122_sat_13.jpg', 'train/102122_sat_14.jpg', 'train/102122_sat_15.jpg', 'train/10233_sat_0.jpg', 'train/10233_sat_1.jpg', 'train/10233_sat_2.jpg', 'train/10233_sat_3.jpg', 'train/10233_sat_4.jpg', 'train/10233_sat_5.jpg', 

In [12]:
class SatelliteDataset(torch.utils.data.Dataset):
    def __init__(self, image_paths, mask_paths, transform=None):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        mask_path = self.mask_paths[idx]

        # Load image and mask
        image = Image.open(image_path)
        mask = Image.open(mask_path)

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        return image, mask


In [13]:

train_list = [os.path.join(base_dir, path) for path in train_list]
train_mask = [os.path.join(base_dir, path) for path in train_mask]
valid_list = [os.path.join(base_dir, path) for path in valid_list]
valid_mask = [os.path.join(base_dir, path) for path in valid_mask]

train_dataset = SatelliteDataset(train_list, train_mask, transform=transforms.ToTensor())
valid_dataset = SatelliteDataset(valid_list, valid_mask, transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=32, shuffle=False)


In [14]:
class ANN(nn.Module):
    def __init__(self, input_size):
        super(ANN, self).__init__()
        self.name = "ANN"
        self.layer1 = nn.Linear(input_size, 60)
        self.layer2 = nn.Linear(60, 1)

    def forward(self, x):
        flattened = x.view(x.size(0), -1)
        activation1 = self.layer1(flattened)
        activation1 = F.relu(activation1)
        activation2 = self.layer2(activation1)
        x = activation2.squeeze(1) # Flatten to [batch_size]
        return x



In [15]:
# Training
def get_model_name(name, batch_size, learning_rate, epoch):
    path = "model_{0}_bs{1}_lr{2}_epoch{3}".format(name, batch_size, learning_rate, epoch)
    return path

def normalize_label(labels):
    max_val = torch.max(labels)
    min_val = torch.min(labels)
    norm_labels = (labels - min_val)/(max_val - min_val)
    return norm_labels

def evaluate(net, loader, criterion):
    total_loss = 0.0
    corr = 0.0
    total = 0
    for i, data in enumerate(loader, 0):
        inputs, labels = data
        outputs = net(inputs)

        predict = outputs.max(1, keepdim=True)[1]
        corr += predict.eq(labels.view_as(predict)).sum().item()

        loss = criterion(outputs, labels)

        total += inputs.shape[0]
        total_loss += loss.item()

    error = 1- (corr / total)
    losses = float(total_loss) / len(loader)
    return error, losses

In [16]:
def train_ann(net, train_loader, valid_loader, criterion, optimizer, num_epochs=10, batch_size=32, learning_rate=0.001):
    train_errs = []
    train_losses = []
    val_errs = []
    val_losses = []

    for epoch in range(num_epochs):
        start_time = time.time()
        net.train()
        running_loss = 0.0
        total_corr = 0
        total_images = 0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_corr += (predicted == labels).sum().item()
            total_images += labels.size(0)

        train_loss = running_loss / len(train_loader)
        train_acc = total_corr / total_images
        train_err = 1.0 - train_acc

        val_err, val_loss = evaluate(net, valid_loader, criterion)

        print(f"Epoch {epoch+1}/{num_epochs}, "
              f"Train Loss: {train_loss:.4f}, Train Error: {train_err:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Error: {val_err:.4f}, "
              f"Time: {time.time() - start_time:.2f}s")

        train_errs.append(train_err)
        train_losses.append(train_loss)
        val_errs.append(val_err)
        val_losses.append(val_loss)

        # Save the model
        model_path = get_model_name(net.name, batch_size, learning_rate, epoch)
        torch.save(net.state_dict(), model_path)

        # Save the training/validation errors and losses
        np.savetxt(f"{model_path}_train_err.csv", train_errs)
        np.savetxt(f"{model_path}_train_loss.csv", train_losses)
        np.savetxt(f"{model_path}_val_err.csv", val_errs)
        np.savetxt(f"{model_path}_val_loss.csv", val_losses)

    return train_errs, train_losses, val_errs, val_losses


In [17]:
###############################################################################
# Training Curve
def plot_training_curve(path):
    import matplotlib.pyplot as plt
    train_err = np.loadtxt("{}_train_err.csv".format(path))
    val_err = np.loadtxt("{}_val_err.csv".format(path))
    train_loss = np.loadtxt("{}_train_loss.csv".format(path))
    val_loss = np.loadtxt("{}_val_loss.csv".format(path))
    plt.title("Train vs Validation Error")
    n = len(train_err) # number of epochs
    plt.plot(range(1,n+1), train_err, label="Train")
    plt.plot(range(1,n+1), val_err, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Error")
    plt.legend(loc='best')
    plt.show()
    plt.title("Train vs Validation Loss")
    plt.plot(range(1,n+1), train_loss, label="Train")
    plt.plot(range(1,n+1), val_loss, label="Validation")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend(loc='best')
    plt.show()

In [18]:
class SatelliteDataset(torch.utils.data.Dataset):
    def __init__(self, image_paths, mask_paths, transform=None):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        mask_path = self.mask_paths[idx]

        # Load image and mask
        image = Image.open(image_path)
        mask = Image.open(mask_path)

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)

        return image, mask

# Assuming train_list, train_mask, valid_list, valid_mask are lists of image and mask paths
train_dataset = SatelliteDataset(train_list, train_mask, transform=transforms.ToTensor())
valid_dataset = SatelliteDataset(valid_list, valid_mask, transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=32, shuffle=False)


In [21]:

input_size = 612 * 612 * 3
net = ANN(input_size)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

# Assuming you have DataLoader objects train_loader and valid_loader
train_errs, train_losses, val_errs, val_losses = train_ann(net, train_loader, valid_loader, criterion, optimizer, num_epochs=10, batch_size=32, learning_rate=0.001)


RuntimeError: 0D or 1D target tensor expected, multi-target not supported